1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)


In [372]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [373]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [374]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=26)

In [375]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [376]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [377]:
scaler = MinMaxScaler()

In [378]:
# scaler.fit(np.vstack((X_train[continuous_columns], X_test[continuous_columns])))

In [379]:
final_transformers = list()
# scaler = MinMaxScaler()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))

In [380]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [381]:
pipelineRF = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 26))
    
])
pipelineRF.fit(X_train, y_train)
preds = pipelineRF.predict_proba(X_test)[:, 1]

In [382]:
precisionRF, recallRF, thresholdsRF = precision_recall_curve(y_test, preds)

fscoreRF = (2 * precisionRF * recallRF) / (precisionRF + recallRF)
ix = np.argmax(fscoreRF)
print(f'Best Threshold={thresholdsRF[ix]}, F-Score={fscoreRF[ix]:.3f}, Precision={precisionRF[ix]:.3f}, Recall={recallRF[ix]:.3f}')

Best Threshold=0.35, F-Score=0.630, Precision=0.610, Recall=0.651


In [383]:
results = pd.DataFrame({'model': ['RandomForest'],
                        'Fscore': [fscoreRF[ix]],
              'precision': [precisionRF[ix]],
              'recall': [recallRF[ix]],
              'threshold': [thresholdsRF[ix]]})

In [384]:
pipelineGB = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 26)),
])
pipelineGB.fit(X_train, y_train)
preds = pipelineGB.predict_proba(X_test)[:, 1]

In [385]:
precisionGB, recallGB, thresholdsGB = precision_recall_curve(y_test, preds)

fscoreGB = (2 * precisionGB * recallGB) / (precisionGB + recallGB)
ix = np.argmax(fscoreGB)
print(f'Best Threshold={thresholdsGB[ix]}, F-Score={fscoreGB[ix]:.3f}, Precision={precisionGB[ix]:.3f}, Recall={recallGB[ix]:.3f}')

Best Threshold=0.2923309979838839, F-Score=0.645, Precision=0.607, Recall=0.687


In [386]:
results = results.append({'model': 'GradBoosting',
                          'Fscore': fscoreGB[ix],
              'precision': precisionGB[ix],
              'recall': recallGB[ix],
              'threshold': thresholdsGB[ix]}, ignore_index=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1900\2459243586.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model': 'GradBoosting',


In [387]:
pipelineLR = Pipeline([
    ('features',feats),
    ('scaler', scaler),
    ('classifier', LogisticRegression(random_state = 26)),
])
pipelineLR.fit(X_train, y_train)
preds = pipelineLR.predict_proba(X_test)[:, 1]

In [388]:
precisionLR, recallLR, thresholdsLR = precision_recall_curve(y_test, preds)

fscoreLR = (2 * precisionLR * recallLR) / (precisionLR + recallLR)
fscoreLR = np.nan_to_num(fscoreLR)
ix = np.argmax(fscoreLR)
print(f'Best Threshold={thresholdsLR[ix]}, F-Score={fscoreLR[ix]:.3f}, Precision={precisionLR[ix]:.3f}, Recall={recallLR[ix]:.3f}')

Best Threshold=0.19827086071182923, F-Score=0.480, Precision=0.357, Recall=0.732


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1900\1728934764.py:3: RuntimeWarning: invalid value encountered in true_divide
  fscoreLR = (2 * precisionLR * recallLR) / (precisionLR + recallLR)


In [389]:
results = results.append({'model': 'LogReg',
              'Fscore': fscoreLR[ix],
              'precision': precisionLR[ix],
              'recall': recallLR[ix],
              'threshold': thresholdsLR[ix]}, ignore_index=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1900\2724780603.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model': 'LogReg',


2.Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)

In [390]:
results

,model,Fscore,precision,recall,threshold
0,RandomForest,0.629890,0.609709,0.651452,0.350000
1,GradBoosting,0.644596,0.607339,0.686722,0.292331
2,LogReg,0.479946,0.356926,0.732365,0.198271


*По метрикам сильно отстает LogReg, лучшая модель - GradBoost. Тут лучше смотреть на fscore. С одной стороны нам важнее полнота: лучше мы позвоним клиентам, которые не собираются уходить, но точно охватим всех, кого надо переубедить. С другой стороны, мы можем лишний раз побеспокоить тех, кто не собирался уходить, и они передумают.<br>
Но если рассуждать с экономической точки зрения, наверное нам важнее точность, чтобы не тратить деньги на тех, кто и так остаётся.*

3.Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

In [391]:
pipelineGB.fit(X_train, y_train)
preds = pipelineGB.predict_proba(X_test)[:, 1]

precisionGB, recallGB, thresholdsGB = precision_recall_curve(y_test, preds)

fscoreGB = (2 * precisionGB * recallGB) / (precisionGB + recallGB)
ix = np.argmax(fscoreGB)
print(f'Best Threshold={thresholdsGB[ix]}, F-Score={fscoreGB[ix]:.3f}, Precision={precisionGB[ix]:.3f}, Recall={recallGB[ix]:.3f}')

Best Threshold=0.2923309979838839, F-Score=0.645, Precision=0.607, Recall=0.687


In [392]:
cnf_matrix = confusion_matrix(y_test, preds>thresholdsGB[ix])
cnf_matrix

array([[1805,  213],
       [ 152,  330]], dtype=int64)

In [393]:
savings = 2*cnf_matrix[1,1]-cnf_matrix[0,1]-cnf_matrix[1,1]
savings

117

По предсказаниям модели выгода составит 117$.<br>
Попробуем перебрать отсечки вручную:

In [394]:
thresh = np.linspace(0,1,100)
savs = []
for i in thresh:
    cnf_matrix = confusion_matrix(y_test, preds>i)
    savs.append(2*cnf_matrix[1,1]-cnf_matrix[0,1]-cnf_matrix[1,1])
ix = np.argmax(savs)
best_thr = thresh[ix]
tot_sav = savs[ix]
print(f'Best threshold: {best_thr:.4f}, savings: {tot_sav}')

Best threshold: 0.5859, savings: 182


На самом деле, лучшая отсечка с экономической точки зрения выше и выгода при ней составит аж 182$


4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3


In [399]:
params={'classifier__max_features':[0.2, 0.3, 0.5],
        'classifier__min_samples_leaf':[2, 3, 5],
        'classifier__max_depth':[4, 5, 7],
        'classifier__min_samples_split':[2, 3, 4]
        }

In [400]:
grid = GridSearchCV(pipelineGB,
                    param_grid=params,
                    cv=6,
                    refit=False,
                   scoring='precision')

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_depth': 4,
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 3,
 'classifier__min_samples_split': 2}

In [401]:
pipelineGB = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 26,
                                              max_depth=4, max_features=0.3, min_samples_leaf=3, min_samples_split=2)),
])
pipelineGB.fit(X_train, y_train)
preds = pipelineGB.predict_proba(X_test)[:, 1]

In [402]:
precisionGB, recallGB, thresholdsGB = precision_recall_curve(y_test, preds)

fscoreGB = (2 * precisionGB * recallGB) / (precisionGB + recallGB)
ix = np.argmax(fscoreGB)
print(f'Best Threshold={thresholdsGB[ix]}, F-Score={fscoreGB[ix]:.3f}, Precision={precisionGB[ix]:.3f}, Recall={recallGB[ix]:.3f}')

Best Threshold=0.3482726588635145, F-Score=0.649, Precision=0.650, Recall=0.647


In [403]:
results = results.append({'model': 'GradBoostingBestParams',
                          'Fscore': fscoreGB[ix],
              'precision': precisionGB[ix],
              'recall': recallGB[ix],
              'threshold': thresholdsGB[ix]}, ignore_index=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1900\2846011137.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'model': 'GradBoostingBestParams',


In [404]:
results

,model,Fscore,precision,recall,threshold
0,RandomForest,0.629890,0.609709,0.651452,0.350000
1,GradBoosting,0.644596,0.607339,0.686722,0.292331
2,LogReg,0.479946,0.356926,0.732365,0.198271
3,GradBoostingBestParams,0.648649,0.650000,0.647303,0.348273


5. (опционально) Еще раз провести оценку экономической эффективности

In [405]:
cnf_matrix = confusion_matrix(y_test, preds>thresholdsGB[ix])
savings = 2*cnf_matrix[1,1]-cnf_matrix[0,1]-cnf_matrix[1,1]
savings

143

После подбора параметров через GridSearch выручка вышла аж 143 \\$ (до подбора было 117 \\$ ). Хотя параметры изменились не сильно значительно (recall вообще просел, значит precision точно важнее, раз он стал лучше и выручка выросла)<br>
Попробуем ещё раз подобрать отсечку вручную, чтобы подтянуть бизнесовый показатель, а не DS-метрику 

In [406]:
thresh = np.linspace(0,1,100)
savs = []
for i in thresh:
    cnf_matrix = confusion_matrix(y_test, preds>i)
    savs.append(2*cnf_matrix[1,1]-cnf_matrix[0,1]-cnf_matrix[1,1])
ix = np.argmax(savs)
best_thr = thresh[ix]
tot_sav = savs[ix]
print(f'Best threshold: {best_thr:.4f}, savings: {tot_sav}')

Best threshold: 0.5152, savings: 174


С этой моделью выручка вышла 174\\$ (хотя раньше удавалось подобрать 182$). Возможно статистически разница незначимая (нужно проверить), но в целом есть смысл попробовать подобрать др.параметры (мб поменять скоринг в GridSearch)